# Recommender Systems
    By Oghosa Igbinakenzua
    MIE 451 | Decision Support Systems
    November 5, 2017
    
This project involves creating a recommender engine, evaluating it to understand its performance, and making changes to improve performance.

## Imports

In [1]:
# import required libraries
import os
import os.path
import numpy as np
import pandas as pd
from math import sqrt
from heapq import nlargest
from tqdm import trange
from tqdm import tqdm
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.metrics import mean_squared_error

## Support functions and variables

In [2]:
#!unzip ml-100k.zip -d .

Archive:  ml-100k.zip
   creating: ./ml-100k/
  inflating: ./ml-100k/allbut.pl     
  inflating: ./ml-100k/mku.sh        
  inflating: ./ml-100k/README        
  inflating: ./ml-100k/u.data        
  inflating: ./ml-100k/u.genre       
  inflating: ./ml-100k/u.info        
  inflating: ./ml-100k/u.item        
  inflating: ./ml-100k/u.occupation  
  inflating: ./ml-100k/u.user        
  inflating: ./ml-100k/u1.base       
  inflating: ./ml-100k/u1.test       
  inflating: ./ml-100k/u2.base       
  inflating: ./ml-100k/u2.test       
  inflating: ./ml-100k/u3.base       
  inflating: ./ml-100k/u3.test       
  inflating: ./ml-100k/u4.base       
  inflating: ./ml-100k/u4.test       
  inflating: ./ml-100k/u5.base       
  inflating: ./ml-100k/u5.test       
  inflating: ./ml-100k/ua.base       
  inflating: ./ml-100k/ua.test       
  inflating: ./ml-100k/ub.base       
  inflating: ./ml-100k/ub.test       


In [3]:
MOVIELENS_DIR = "ml-100k"

In [4]:
!ls {MOVIELENS_DIR}

README       u.genre      u.user       u2.test      u4.test      ua.test
allbut.pl    u.info       u1.base      u3.base      u5.base      ub.base
mku.sh       u.item       u1.test      u3.test      u5.test      ub.test
u.data       u.occupation u2.base      u4.base      ua.base


In [7]:
def getData(folder_path, file_name):
    fields = ['userID', 'itemID', 'rating', 'timestamp']
    data = pd.read_csv(os.path.join(folder_path, file_name), sep='\t', names=fields)
    return data 

In [8]:
rating_df = getData(MOVIELENS_DIR, 'u.data')

In [9]:
num_users = len(rating_df.userID.unique())
num_items = len(rating_df.itemID.unique())
print("Number of users:", num_users)
print("Number of items:", num_items)

Number of users: 943
Number of items: 1682


## Q1

### (a)

In [16]:
def dataPreprocessor(rating_df, num_users, num_items):
    """
        INPUT: 
            data: pandas DataFrame. columns=['userID', 'itemID', 'rating' ...]
            num_row: int. number of users
            num_col: int. number of items
            
        OUTPUT:
            matrix: 2D numpy array. row IDs are (userID-1), columns IDs are (itemID-1),
            and the rating for (userID,itemID,rating) is the value at this row and column.  
            Any observed ratings are zero.
            
        NOTE 1: see where something very similar is done in the lab in function 'buildUserItemMatrix'    
            
        NOTE 2: data can have more columns, but your function should ignore 
              additional columns.
              
    """
    matrix = np.zeros((num_users, num_items))
    ########### your code goes here ###########
    for (index, userID, itemID, rating, timestamp) in rating_df.itertuples():
        matrix[userID-1, itemID-1] = rating  
    
    ###########         end         ###########
    return matrix

In [17]:
dataPreprocessor(rating_df, num_users, num_items)

array([[ 5.,  3.,  4., ...,  0.,  0.,  0.],
       [ 4.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 5.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  5.,  0., ...,  0.,  0.,  0.]])

### (b)

In [30]:
class BaseLineRecSys(object):
    def __init__(self, method, processor=dataPreprocessor):
        """
            method: string. From ['popularity','useraverage']
            processor: function name. dataPreprocessor by default
        """
        self.method_name = method
        self.method = self._getMethod(self.method_name)
        self.processor = processor
        self.pred_column_name = self.method_name
        
    def _getMethod(self, method_name):
        """
            Don't change this
        """
        switcher = {
            'popularity': self.popularity,
            'useraverage': self.useraverage,
        }
        
        return switcher[method_name]
    
    @staticmethod
    def useraverage(train_matrix, num_users, num_items):
        """
            INPUT:
                train_matrix: 2D numpy array.
                num_users: int. Number of Users.
                num_items: int. Number of Items.
            OUTPUT:
                predictionMatrix: 2D numpy array. this is the same dimensions and 
                row/column IDs as train_matrix, but anywhere there is a 0 in train_matrix, 
                there should be a predicted value in predictedMatrix.
                
            NOTE: see where something very similar is done in the lab in function 'predictByUserAverage'  
            
        """
        
        predictionMatrix = np.zeros((num_users, num_items))
        ########### your code goes here ###########
        
        for (user,item), rating in np.ndenumerate(train_matrix):
            # Predict rating for every item that wasn't ranked by the user (rating == 0)
            # Extract the items the user already rated
            userVector = train_matrix[user, :]
            ratedItems = userVector[userVector.nonzero()]
            
            # If not empty, calculate average and set as rating for the current item
            if ratedItems.size == 0:
                itemAvg = 0
            else:
                itemAvg = ratedItems.mean()
            predictionMatrix[user, item] = itemAvg
            
            # report progress every 100 users
            if (user % 100 == 0 and item == 1):
                print ("calculated %d users" % (user,))
        

        ###########         end         ###########
        return predictionMatrix
    
    @staticmethod
    def popularity(train_matrix, num_users, num_items):
        """
            INPUT:
                train_matrix: 2D numpy array.
                num_users: int. Number of Users.
                num_items: int. Number of Items.
            OUTPUT:
                predictionMatrix: 2D numpy array. this is the same dimensions and 
                row/column IDs as train_matrix, but anywhere there is a 0 in train_matrix, 
                there should be a predicted value in predictedMatrix.
                
            NOTE: see where something very similar is done in the lab in function 'predictByPopularity'    
        """
        
        predictionMatrix = np.zeros((num_users, num_items))
        ########### your code goes here ###########

        # Define function for converting 1-5 rating to 0/1 (like / don't like)
        vf = np.vectorize(lambda x: 1 if x >= 4 else 0)
    
        # For every item calculate the number of people liked (4-5) divided by the number of people that rated
        itemPopularity = np.zeros((num_items))
        sumOfItemsRated = 0
        for item in range(num_items):
            numOfUsersRated = len(train_matrix[:, item].nonzero()[0])
            sumOfItemsRated += numOfUsersRated
            numOfUsersLiked = len(vf(train_matrix[:, item]).nonzero()[0])
            if numOfUsersRated == 0:
                itemPopularity[item] = 0
            else:
                itemPopularity[item] = numOfUsersLiked/numOfUsersRated
                
        for (user,item), rating in np.ndenumerate(train_matrix):
            # Predict rating for every item that wasn't ranked by the user (rating == 0)
            #if rating == 0:
            predictionMatrix[user, item] = itemPopularity[item]
            
            # report progress every 100 users
            if (user % 100 == 0 and item == 1):
                print ("calculated %d users" % (user,))
        
                
        ###########         end         ###########
        return predictionMatrix    
    
    def predict_all(self, train_df, num_users, num_items):
        
        train_matrix = self.processor(train_df, num_users, num_items)
        self.__model = self.method(train_matrix, num_users, num_items)
        
    def evaluate_test(self, test_df, copy=False):
        
        if copy:
            prediction = test_df.copy()
        else:
            prediction = test_df
            
        prediction[self.pred_column_name] = np.nan
        
        for (index, 
             userID, 
             itemID) in tqdm(prediction[['userID','itemID']].itertuples()):
            prediction.ix[index, self.pred_column_name] = self.__model[userID-1, itemID-1]

        return prediction
        
    def getModel(self):
        """
            return predicted user-item matrix
        """
        return self.__model
    
    def getPredColName(self):
        """
            return prediction column name
        """
        return self.pred_column_name
    
    def reset(self):
        """
            reuse the instance of the class by removing model
        """
        try:
            self.model = None
        except:
            print("You don not have model..")
            

In [31]:
popularity_recsys = BaseLineRecSys('popularity')

In [32]:
popularity_recsys.predict_all(rating_df, num_users, num_items)

calculated 0 users
calculated 100 users
calculated 200 users
calculated 300 users
calculated 400 users
calculated 500 users
calculated 600 users
calculated 700 users
calculated 800 users
calculated 900 users


In [33]:
popularity_recsys.getModel()

array([[ 0.71017699,  0.38931298,  0.37777778, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.71017699,  0.38931298,  0.37777778, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.71017699,  0.38931298,  0.37777778, ...,  0.        ,
         0.        ,  0.        ],
       ..., 
       [ 0.71017699,  0.38931298,  0.37777778, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.71017699,  0.38931298,  0.37777778, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.71017699,  0.38931298,  0.37777778, ...,  0.        ,
         0.        ,  0.        ]])

In [34]:
rating_df.head()

,userID,itemID,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [25]:
popularity_recsys.evaluate_test(rating_df,copy=True).head()

0it [00:00, ?it/s]/Users/oghosa/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:130: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
100000it [01:02, 1609.76it/s]


,userID,itemID,rating,timestamp,popularity
0,196,242,3,881250949,0.760684
1,186,302,3,891717742,0.804714
2,22,377,1,878887116,0.076923
3,244,51,2,880606923,0.555556
4,166,346,1,886397596,0.611111


In [35]:
average_user_rating_recsys = BaseLineRecSys('useraverage')

In [36]:
average_user_rating_recsys.predict_all(rating_df, num_users, num_items)

calculated 0 users
calculated 100 users
calculated 200 users
calculated 300 users
calculated 400 users
calculated 500 users
calculated 600 users
calculated 700 users
calculated 800 users
calculated 900 users


In [37]:
average_user_rating_recsys.getModel()

array([[ 3.61029412,  3.61029412,  3.61029412, ...,  3.61029412,
         3.61029412,  3.61029412],
       [ 3.70967742,  3.70967742,  3.70967742, ...,  3.70967742,
         3.70967742,  3.70967742],
       [ 2.7962963 ,  2.7962963 ,  2.7962963 , ...,  2.7962963 ,
         2.7962963 ,  2.7962963 ],
       ..., 
       [ 4.04545455,  4.04545455,  4.04545455, ...,  4.04545455,
         4.04545455,  4.04545455],
       [ 4.26582278,  4.26582278,  4.26582278, ...,  4.26582278,
         4.26582278,  4.26582278],
       [ 3.41071429,  3.41071429,  3.41071429, ...,  3.41071429,
         3.41071429,  3.41071429]])

In [38]:
average_user_rating_recsys.evaluate_test(rating_df,copy=True).head()

0it [00:00, ?it/s]/Users/oghosa/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:126: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
100000it [01:01, 1620.92it/s]


,userID,itemID,rating,timestamp,useraverage
0,196,242,3,881250949,3.615385
1,186,302,3,891717742,3.413043
2,22,377,1,878887116,3.351562
3,244,51,2,880606923,3.651261
4,166,346,1,886397596,3.550000


## Q2

### (a)

In [52]:
class SimBasedRecSys(object):

    def __init__(self, base, method, processor=dataPreprocessor):
        """
            base: string. From ['user', 'item']. User-based Similarity or Item-based
            method: string. From ['cosine', 'euclidean', 'somethingelse']
            processor: function name. dataPreprocessor by default
        """
        self.base = base
        self.method_name = method
        self.method = self._getMethod(self.method_name)
        self.processor = processor
        self.pred_column_name = self.base+'-'+self.method_name
    
    def _getMethod(self, method_name):
        """
            Don't change this
        """
        switcher = {
            'cosine': self.cosine,
            'euclidean': self.euclidean,
            'somethingelse': self.somethingelse,
        }
        
        return switcher[method_name]
    
    @staticmethod
    def cosine(matrix):
        """
            cosine similarity
        """
        similarity_matrix = 1 - pairwise_distances(matrix, metric='cosine')
        return similarity_matrix
    
    @staticmethod
    def euclidean(matrix):
        """
            euclidean similarity
        
        INPUT
            matrix: same as the rating matrix generated by dataPreprocessor 
            with R rows and C columns.  Outputs an R x R similarity_matrix S 
            where each S_ij should be the euclidean similarity between row i and 
            row j of matrix.
        """
        ########### your code goes here ###########

        similarity_matrix = 1 / (1 + pairwise_distances(matrix, metric='euclidean'))

    
        ###########         end         ###########    
        
        return similarity_matrix
    
    @staticmethod
    def somethingelse(matrix):
        """
            manhattan? or super-natural intuition similarity
            
        INPUT
            matrix: same as the rating matrix generated by dataPreprocessor 
            with R rows and C columns.  Outputs an R x R similarity_matrix S 
            where each S_ij should be the somethingelse similarity between row i and 
            row j of matrix.
        """
        ########### your code goes here ###########
    
    
        similarity_matrix = 1 / (1 + pairwise_distances(matrix, metric='manhattan'))

        ###########         end         ###########        
        return similarity_matrix
        
    def predict_all(self, train_df, num_users, num_items):
        """
            INPUT: 
                data: pandas DataFrame. columns=['userID', 'itemID', 'rating'...]
                num_row: scalar. number of users
                num_col: scalar. number of items
            OUTPUT:
                no return... this method assigns the result to self.__model
                
                self.__model: this is the same dimensions and row/column IDs as train_matrix, 
                but anywhere there is a 0 in train_matrix, there should be a predicted value 
                in self.__model.
            
            NOTES:
                self.__model should contain predictions for *all* user and items
                (don't worry about predicting for observed (user,item) pairs,
                 since we won't be using these predictions in the evaluation)
                (see 'vectorizedUserSimRecSys' code in for an efficient vectorized example)
                
        """
        train_matrix = self.processor(train_df, num_users, num_items)
        
        
        temp_matrix = np.zeros(train_matrix.shape)
        temp_matrix[train_matrix.nonzero()] = 1
        if self.base == 'user':
            ########### your code goes here ###########

            uu_similarity = self.method(train_matrix)
            normalizer = np.matmul(uu_similarity, temp_matrix)
            normalizer[normalizer == 0] = 1e-5
            self.__model = np.matmul(uu_similarity, train_matrix)/normalizer
            
            # average each user's score over all the movies
            useraverage = np.sum(train_matrix, axis=1)/(np.sum(temp_matrix, axis=1) + 1e-5)
            # sum each movies over all the users 
            columns = np.sum(self.__model, axis=0)
            # if a movie didn't get any rankings/scores add 
            self.__model[:, columns==0] = self.__model[:, columns==0] + np.expand_dims(useraverage, axis=1)
          
  
            
            
            ###########         end         ###########
            
        elif self.base == 'item':
            ########### your code goes here ###########

            ii_similarity = self.method(np.transpose(train_matrix))
            normalizer = np.matmul(temp_matrix, ii_similarity)
            normalizer[normalizer == 0] = 1e-5
            self.__model = np.matmul(train_matrix, ii_similarity)/normalizer
            
            itemaverage = np.sum(train_matrix, axis=1)/(np.sum(temp_matrix, axis=1) + 1e-5)
            columns = np.sum(self.__model, axis=0)
            self.__model[:, columns==0] = self.__model[:, columns==0] + np.expand_dims(itemaverage, axis=1)
            
           
            
            
            ###########         end         ###########
        else:
            print('No other option available')
        
        
    def evaluate_test(self, test_df, copy=False):
        """
            INPUT:
                data: pandas DataFrame. columns=['userID', 'itemID', 'rating'...]
            OUTPUT:
                predictions:  pandas DataFrame. 
                              columns=['userID', 'itemID', 'rating', 'base-method'...]
                              
            NOTE: 1. data can have more columns, but your function should ignore 
                  additional columns.
                  2. 'base-method' depends on your 'base' and 'method'. For example,
                  if base == 'user' and method == 'cosine', 
                  then base-method == 'user-cosine'
                  3. your predictions go to 'base-method' column
        """
        if copy:
            prediction = test_df.copy()
        else:
            prediction = test_df
        prediction[self.pred_column_name] = np.nan
        
        for (index, 
             userID, 
             itemID) in tqdm(prediction[['userID','itemID']].itertuples()):
            prediction.ix[index, self.pred_column_name] = self.__model[userID-1, itemID-1]
    
        return prediction
    
    def getModel(self):
        """
            return predicted user-item matrix
        """
        return self.__model
    
    def getPredColName(self):
        """
            return prediction column name
        """
        return self.pred_column_name
    
    def reset(self):
        """
            reuse the instance of the class by removing model
        """
        try:
            self.model = None
        except:
            print("You do not have model..")

In [42]:
# Examples of how to call similarity functions.
I = np.eye(3)
SimBasedRecSys.cosine(I)

array([[ 1.,  0.,  0.],
       [ 0.,  1.,  0.],
       [ 0.,  0.,  1.]])

In [43]:
SimBasedRecSys.euclidean(I)

array([[ 1.        ,  0.41421356,  0.41421356],
       [ 0.41421356,  1.        ,  0.41421356],
       [ 0.41421356,  0.41421356,  1.        ]])

In [44]:
SimBasedRecSys.somethingelse(I)

array([[ 1.        ,  0.33333333,  0.33333333],
       [ 0.33333333,  1.        ,  0.33333333],
       [ 0.33333333,  0.33333333,  1.        ]])

Cosine metrics works better beacause it uses the direction (angle) beteween the vectors to measure the similarity rather than the distance between the points which could be strongly biased based on the magnitude of the vector.

### (b)

I used the 'Manhattan' metric beacause since it is an absolute value distance, it should give more robust results, whereas Euclidean would be influenced by unusual values.

## Q3

### (a)

In [53]:
user_cosine_recsys = SimBasedRecSys('user','cosine')

In [54]:
user_cosine_recsys.predict_all(rating_df, num_users, num_items)

In [55]:
user_cosine_recsys.getModel()

array([[ 3.89911175,  3.19022667,  3.0261129 , ...,  2.        ,
         3.        ,  3.        ],
       [ 3.84034456,  3.17139889,  2.92626717, ...,  2.        ,
         3.        ,  3.        ],
       [ 3.87104065,  3.12823798,  3.03250708, ...,  2.        ,
         3.        ,  3.        ],
       ..., 
       [ 3.90754645,  3.20227238,  3.05776201, ...,  2.        ,
         3.        ,  3.        ],
       [ 3.91100649,  3.21591021,  2.98854017, ...,  2.        ,
         3.        ,  3.        ],
       [ 3.91593122,  3.24268207,  3.08255897, ...,  0.        ,
         3.        ,  3.        ]])

In [56]:
rating_df.head()

,userID,itemID,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [57]:
user_cosine_recsys.evaluate_test(rating_df,copy=True).head()

0it [00:00, ?it/s]/Users/oghosa/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:163: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
100000it [01:05, 1517.00it/s]


,userID,itemID,rating,timestamp,user-cosine
0,196,242,3,881250949,4.025213
1,186,302,3,891717742,4.142828
2,22,377,1,878887116,1.922080
3,244,51,2,880606923,3.431884
4,166,346,1,886397596,3.424963


In [58]:
item_cosine_recsys = SimBasedRecSys('item','cosine')
item_cosine_recsys.predict_all(rating_df, num_users, num_items)
item_cosine_recsys.getModel()

array([[ 3.75429099,  3.66419957,  3.73222997, ...,  3.60248287,
         3.79662696,  3.90232044],
       [ 3.83658867,  3.80424519,  3.77473905, ...,  3.72798332,
         3.9109779 ,  3.79775927],
       [ 2.84492718,  2.89389328,  2.84327324, ...,  2.99504451,
         3.16444153,  2.9858119 ],
       ..., 
       [ 4.11427954,  4.0558267 ,  4.00963139, ...,  4.        ,
         3.87872799,  4.14814803],
       [ 4.37096823,  4.39679254,  4.33543016, ...,  3.955358  ,
         4.41891089,  4.57995134],
       [ 3.52030345,  3.46948821,  3.52393064, ...,  0.        ,
         3.6110641 ,  3.59656861]])

In [59]:
item_cosine_recsys.evaluate_test(rating_df,copy=True).head()

0it [00:00, ?it/s]/Users/oghosa/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:163: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
100000it [01:04, 1543.60it/s]


,userID,itemID,rating,timestamp,item-cosine
0,196,242,3,881250949,3.591314
1,186,302,3,891717742,3.344077
2,22,377,1,878887116,2.965365
3,244,51,2,880606923,3.637332
4,166,346,1,886397596,3.333013


### (b)

In [60]:
class CrossValidation(object):
    def __init__(self, metric, data_path=MOVIELENS_DIR):
        """
            INPUT:
                metric: string. from['RMSE','P@K','R@K']
        """
        self.folds = self._getData(MOVIELENS_DIR)
        self.metric_name = metric
        self.metric = self._getMetric(self.metric_name)
        
    def _getMetric(self, metric_name):
        """
            Don't change this
        """
        switcher = {
            'RMSE': self.rmse,
            'P@K': self.patk,
            'R@K': self.ratk,
        }
        
        return switcher[metric_name]
    
    @staticmethod
    def rmse(data, k, num_users, num_items, pred, true='rating'):
        """
            data: pandas DataFrame. 
            pred: string. Column name that corresponding to the prediction
            true: string. Column name that corresponding to the true rating
        """
        return sqrt(mean_squared_error(data[pred], data[true]))
    
    # Precision at k
    def patk(self, data, k, num_users, num_items, pred, true='rating'):
        """
            data: pandas DataFrame. 
            k: top-k items retrived
            pred: string. Column name that corresponding to the prediction
            true: string. Column name that corresponding to the true rating
        """
        prediction = self.getMatrix(data, num_users, num_items, pred)
        testSet =  self.getMatrix(data, num_users, num_items, true)
    
        # Initialize sum and count vars for average calculation
        sumPrecisions = 0
        countPrecisions = 0

        # Define function for converting 1-5 rating to 0/1 (like / don't like)
        vf = np.vectorize(lambda x: 1 if x >= 4 else 0)

        for userID in range(num_users):
            # Pick top K based on predicted rating
            userVector = prediction[userID,:]
            topK = nlargest(k, range(len(userVector)), userVector.take)

            # Convert test set ratings to like / don't like
            userTestVector = vf(testSet[userID,:]).nonzero()[0]

            # Calculate precision
            precision = len([item for item in topK if item in userTestVector])/len(topK)

            # Update sum and count
            sumPrecisions += precision
            countPrecisions += 1

        # Return average P@k
        return sumPrecisions/countPrecisions
    
    # Recall at k
    def ratk(self, data, k, num_users, num_items, pred, true='rating'):
        """
            data: pandas DataFrame. 
            k: top-k items relevant
            pred: string. Column name that corresponding to the prediction
            true: string. Column name that corresponding to the true rating
        """
        prediction = self.getMatrix(data, num_users, num_items, pred)
        testSet =  self.getMatrix(data, num_users, num_items, true)
        # Initialize sum and count vars for average calculation
        sumRecalls = 0
        countRecalls = 0

        # Define function for converting 1-5 rating to 0/1 (like / don't like)
        vf = np.vectorize(lambda x: 1 if x >= 4 else 0)

        for userID in range(num_users):
            # Pick top K based on predicted rating
            userVector = prediction[userID,:]
            topK = nlargest(k, range(len(userVector)), userVector.take)

            # Convert test set ratings to like / don't like
            userTestVector = vf(testSet[userID,:]).nonzero()[0]

            # Ignore user if has no ratings in the test set
            if (len(userTestVector) == 0):
                continue

            # Calculate recall
            recall = len([item for item in topK if item in userTestVector])/len(userTestVector)

            # Update sum and count
            sumRecalls += recall
            countRecalls += 1

        # Return average R@k
        return sumRecalls/countRecalls
    
    @staticmethod
    def getMatrix(rating_df, num_users, num_items, column_name):
        matrix = np.zeros((num_users, num_items))
    
        for (index, userID, itemID, value) in rating_df[['userID','itemID', column_name]].itertuples():
            matrix[userID-1, itemID-1] = value
            
        return matrix
    
    @staticmethod
    def _getData(data_path):
        """
            Don't change this function
        """
        folds = []
        data_types = ['u{0}.base','u{0}.test']
        for i in range(1,6):
            train_set = getData(data_path, data_types[0].format(i))
            test_set = getData(data_path, data_types[1].format(i))
            folds.append([train_set, test_set])
        return folds
    
    def run(self, algorithms, num_users, num_items, k=1):
        """
            5-fold cross-validation
            algorithms: list. a list of algorithms. 
                        eg: [user_cosine_recsys, item_euclidean_recsys]
        """
        
        scores = {}
        for algorithm in algorithms:
            print('Processing algorithm {0}'.format(algorithm.getPredColName()))
            fold_scores = []
            for fold in self.folds:
                algorithm.reset()
                algorithm.predict_all(fold[0], num_users, num_items)
                prediction = algorithm.evaluate_test(fold[1])
                pred_col = algorithm.getPredColName()
                fold_scores.append(self.metric(prediction, k, num_users, num_items, pred_col))
            scores[algorithm.getPredColName()] = fold_scores
            
        results = scores    
    
        return results
            

In [61]:
# How to use CrossValidation Class?

In [67]:
# 1. gather your algorithms in previous steps.
algorithm_instances = [item_cosine_recsys, 
                       user_cosine_recsys]

In [68]:
# 2. Instantiate a CrossValidation instance and assign the measurement that you want to use
# RMSE, P@K, R@K
# Precision at K in this example
cv_patk = CrossValidation('RMSE')

In [64]:
# 3. Run CV by giving:
#    1> algorithms just gathered
#    2> number of users in the full dataset
#    3> number of items in the full dataset
#    4> precision or recall at K need a K value, so k=5 means precision at 5 in this example
results = cv_patk.run(algorithm_instances, num_users, num_items,k=5)

Processing algorithm user-cosine


0it [00:00, ?it/s]/Users/oghosa/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:163: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
20000it [00:09, 2093.38it/s]
20000it [00:09, 2103.24it/s]
20000it [00:09, 2079.48it/s]
20000it [00:09, 2072.87it/s]
20000it [00:09, 2164.92it/s]


Processing algorithm item-cosine


20000it [00:09, 2120.12it/s]
20000it [00:09, 2116.03it/s]
20000it [00:09, 2129.38it/s]
20000it [00:09, 2162.10it/s]
20000it [00:09, 2194.85it/s]


In [69]:
results

{'item-cosine': [1.033430706071607,
  1.0169957594395471,
  1.0045122528110584,
  1.0099161640556313,
  1.011190718546166],
 'user-cosine': [1.0264490128856898,
  1.0214387664092763,
  1.0132940323507253,
  1.009400399702741,
  1.0161883960008826]}

In [72]:
#Confidence Interval Function
import scipy.stats as stats
from math import sqrt
def mean_confidence_interval(data, confidence=0.95):
    a = 1.0*np.array(data)
    n = len(a)
    mu,sd = np.mean(a),np.std(a)
    z = stats.t.ppf(confidence, n)
    h=z*sd/sqrt(n)
    return mu, h

#print results
for algorithim in results:
    mu, h = mean_confidence_interval(results[algorithim], confidence=0.95)
    print(algorithim)
    print('mean: ', mu)
    print('confidence interval: ', h)
    print('range: (', (mu-h), ', ', (mu+h), ')\n')

user-cosine
mean:  1.01735412147
confidence interval:  0.005414541145
range: ( 1.01193958032 ,  1.02276866261 )

item-cosine
mean:  1.01520912018
confidence interval:  0.00895556216199
range: ( 1.00625355802 ,  1.02416468235 )



The item-item cosine performed better (lower RMSE mean and lower range). A reason for this is because the item-item matix has more item ratings vs user ratings (i.e less sparse) in comparison to the user-user matrix.

It is worth noting that since the 95% CIs overlap the diferences are not statiscaly significant. Running more test is a better way significatly identify a better option.

## Q4

### (a)

In [75]:
algorithm_instances_Q4 = [popularity_recsys, 
                       average_user_rating_recsys, 
                       user_cosine_recsys,
                       item_cosine_recsys]

cv_rmse = CrossValidation('RMSE')
results_rmse = cv_rmse.run(algorithm_instances_Q4, num_users, num_items,k=5)

cv_pk = CrossValidation('P@K')
results_pk = cv_pk.run(algorithm_instances_Q4, num_users, num_items,k=5)

cv_rk = CrossValidation('R@K')
results_rk = cv_rk.run(algorithm_instances_Q4, num_users, num_items,k=5)

Processing algorithm popularity
calculated 0 users
calculated 100 users
calculated 200 users
calculated 300 users
calculated 400 users
calculated 500 users
calculated 600 users
calculated 700 users
calculated 800 users
calculated 900 users

0it [00:00, ?it/s]/Users/oghosa/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:126: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
20000it [00:09, 2089.18it/s]



calculated 0 users
calculated 100 users
calculated 200 users
calculated 300 users
calculated 400 users
calculated 500 users
calculated 600 users
calculated 700 users
calculated 800 users


182it [00:00, 1815.50it/s]

calculated 900 users


...

20000it [00:08, 2233.19it/s]


Processing algorithm user-cosine


20000it [00:09, 2216.57it/s]
20000it [00:09, 2218.91it/s]
20000it [00:08, 2232.10it/s]
20000it [00:08, 2244.25it/s]
20000it [00:09, 2199.50it/s]


Processing algorithm item-cosine


20000it [00:09, 2195.37it/s]
20000it [00:09, 2214.48it/s]
20000it [00:09, 2204.03it/s]
20000it [00:09, 2218.15it/s]
20000it [00:08, 2229.19it/s]


In [79]:
performance_df = pd.DataFrame(columns=['algorithm', 'RMSE Avg', 'RMSE CI', 'P@K Avg', 'P@K CI', 'R@K Avg', 'R@K CI'])

for algorithm in results_rmse:
    mu_rmse, h_rmse = mean_confidence_interval(results_rmse[algorithm], confidence=0.95)
    mu_pk, h_pk = mean_confidence_interval(results_pk[algorithm], confidence=0.95)
    mu_rk, h_rk = mean_confidence_interval(results_rk[algorithm], confidence=0.95)
    performance_df = performance_df.append({'algorithm':algorithm, 'RMSE Avg':mu_rmse, 'RMSE CI':h_rmse, 'P@K Avg':mu_pk, 'P@K CI':h_pk, 'R@K Avg':mu_rk, 'R@K CI':h_rk}, ignore_index=True)

In [80]:
performance_df

,algorithm,RMSE Avg,RMSE CI,P@K Avg,P@K CI,R@K Avg,R@K CI
0,popularity,3.159093,0.012853,0.550583,0.094218,0.484076,0.075910
1,useraverage,1.043718,0.009599,0.473637,0.085452,0.441323,0.072713
2,user-cosine,1.017354,0.005415,0.555843,0.094934,0.486269,0.075834
3,item-cosine,1.015209,0.008956,0.532216,0.096408,0.474971,0.078805


### (b)

Baseline: **Popularity**; Metric: **RMSE**: 

Since the popularity metric returns results on a 0-1 scale (i.e. 1 if movive is well-liked and 0 otherwise) the RMSE metric does not provide a fair comparison.

### (c)

Metric | Best Algorithm | Explantion
------|---------------|-----------
RMSE | Item Cosine | Due to the relatively large amount of item ratings, the accuarys is expected to be highter beacause more feture can be considered.
P@K | User Cosine | This performs best because the cosine metric does not use only magnitude and because data from other users are used to personalize the results
R@K | User Cosine | This performs best because the cosine metric does not use only magnitude and because data from other users are used to personalize the results

### (d)

Yes (exlculding the popularity algorithm due to the RMSE point raised in 4b).

Based on the data, the ranking metrics appear negatively correlated to the RMSE; i.e higher ranking -> lower RMSE. This is reasonable as a higer ranking should imply a lower error from acurratly predicting the user's like/dislike of a movie.

## Q5

### (a)

In [81]:
def item_item_similarity(train_df, num_users, num_items):
    
    train_matrix = dataPreprocessor(train_df, num_users, num_items)
    temp_matrix = np.zeros(train_matrix.shape)
    temp_matrix[train_matrix.nonzero()] = 1

    similarity_matrix = 1 - pairwise_distances(np.transpose(train_matrix), metric='cosine')
    
    return similarity_matrix

In [82]:
fieldsMovies = ['movieID', 'movieTitle', 'releaseDate', 'videoReleaseDate', 'IMDbURL', 'unknown', 'action', 'adventure',
          'animation', 'childrens', 'comedy', 'crime', 'documentary', 'drama', 'fantasy', 'filmNoir', 'horror',
          'musical', 'mystery', 'romance','sciFi', 'thriller', 'war', 'western']
moviesDF = pd.read_csv(os.path.join(MOVIELENS_DIR, 'u.item'), sep='|', names=fieldsMovies, encoding='latin-1')

In [84]:
ctr = 0;
for index in np.argsort(popularity_recsys.getModel()[0]):
    if(ctr!=1000):
        print(moviesDF.loc[index, 'movieID'], moviesDF.loc[index, 'movieTitle'])
        ctr += 1

1682 Scream of Stone (Schrei aus Stein) (1991)
1432 Mighty, The (1998)
1430 Ill Gotten Gains (1997)
1424 I Like It Like That (1994)
1423 Walking Dead, The (1995)
1422 Suture (1993)
1420 Gilligan's Island: The Movie (1998)
1417 Turning, The (1992)
1416 No Escape (1994)
957 Pushing Hands (1992)
1414 Coldblooded (1995)
1412 Land Before Time III: The Time of the Great Giving (1995) (V)
973 Grateful Dead (1995)
1409 Swan Princess, The (1994)
1408 Gordy (1995)
1407 Specialist, The (1994)
424 Children of the Corn: The Gathering (1996)
981 Dangerous Ground (1997)
1371 Machine, The (1994)
1373 Good Morning (1971)
1374 Falling in Love Again (1980)
1378 Rhyme & Reason (1997)
1029 Jury Duty (1995)
1382 Bonheur, Le (1965)
1433 Men of Means (1998)
1383 Second Jungle Book: Mowgli & Baloo, The (1997)
987 Underworld (1997)
1387 Fall (1997)
1389 Mondo (1996)
1390 Innocent Sleep, The (1995)
1393 Stag (1997)
1395 Hurricane Streets (1998)
1384 Squeeze (1996)
1370 I Can't Sleep (J'ai pas sommeil) (1994)
437

1224 Scout, The (1994)
998 Cabin Boy (1994)
386 Addams Family Values (1993)
243 Jungle2Jungle (1997)
586 Terminal Velocity (1994)
892 Flubber (1997)
1178 Major Payne (1994)
982 Maximum Risk (1996)
264 Mimic (1997)
1183 Cowboy Way, The (1994)
938 Smile Like Yours, A (1997)
29 Batman Forever (1995)
1023 Fathers' Day (1997)
1095 High School High (1996)
384 Naked Gun 33 1/3: The Final Insult (1994)
145 Lawnmower Man, The (1992)
231 Batman Returns (1992)
791 Baby-Sitters Club, The (1995)
1376 Meet Wally Sparks (1997)
247 Turbo: A Power Rangers Movie (1997)
1108 Feast of July (1995)
1333 Midnight Dancers (Sibak) (1994)
1031 Lassie (1994)
1088 Double Team (1997)
1527 Senseless (1998)
1346 Dingo (1992)
1324 Loaded (1994)
1470 Gumby: The Movie (1995)
1392 Locusts, The (1997)
263 Steel (1997)
1283 Out to Sea (1997)
978 Heaven's Prisoners (1996)
415 Apple Dumpling Gang, The (1975)
1249 For Love or Money (1993)
1241 Van, The (1996)
994 Last Time I Committed Suicide, The (1997)
407 Spy Hard (1996)


In [86]:
#395-1 Robin Hood: Men in Tights (1993)
#231-1 Batman Returns (1992)
#94-1 Home Alone (1990)

item_similarity = item_item_similarity(rating_df, num_users, num_items)

l = [394, 230, 93]

for index in l:
    print(moviesDF.loc[index, 'movieTitle'])
    similar_movies = ((-item_similarity)[index].argsort()[1:6])
    for movies in similar_movies:
        print('%12s: %12s' % (moviesDF.loc[movies, 'movieID'], moviesDF.loc[movies, 'movieTitle']))
    print()

Robin Hood: Men in Tights (1993)
          94: Home Alone (1990)
          67: Ace Ventura: Pet Detective (1994)
         388: Beverly Hills Cop III (1994)
         386: Addams Family Values (1993)
          80: Hot Shots! Part Deux (1993)

Batman Returns (1992)
         403: Batman (1989)
          29: Batman Forever (1995)
          62: Stargate (1994)
         550: Die Hard: With a Vengeance (1995)
         385: True Lies (1994)

Home Alone (1990)
         393: Mrs. Doubtfire (1993)
          67: Ace Ventura: Pet Detective (1994)
          63: Santa Clause, The (1994)
         403: Batman (1989)
          82: Jurassic Park (1993)



### (b)

Yes. These similaarities are reasonable because the returened movies are indeed similar to the chosen ones. For example the returned movies for batman are other batmane movies. 

## Q6 [GRAD ONLY]

### (a)

# Validation

In [87]:
# Constants for validation only
ROW_NUM = 943
COL_NUM = 1682
RATING_COL = 'rating'

### dataPreprocessor

In [88]:
def testDataPreprocessor(path=MOVIELENS_DIR, getData=getData, getMatrix=CrossValidation.getMatrix):
    validation_df = getData(MOVIELENS_DIR, 'u1.test')
    try:
        matrix = getMatrix(validation_df, ROW_NUM, COL_NUM, RATING_COL)
    except:
        print('dataPreprocessor function has error')
        return
    try:
        assert(matrix.shape == (ROW_NUM,COL_NUM)),\
        "Shape of matrix{0} doesn't match predefined shape (943,1682)".format(matrix.shape)
    except Exception as e:
        print(e)
    return validation_df

In [89]:
validation_df = testDataPreprocessor()

## Baseline Recommendation Systems

### Popularity Based Recommendation

In [90]:
def testPopularityRecSys(validation_df=validation_df, BaseLineRecSys = BaseLineRecSys):
    popularity_recsys = BaseLineRecSys('popularity')
    try:
        popularity_recsys.predict_all(validation_df, ROW_NUM, COL_NUM)
    except Exception as e:        
        print('popularity function has error')
        print(e)
        return
    try:
        predictionMatrix = popularity_recsys.getModel()
        assert(predictionMatrix.shape == (ROW_NUM, COL_NUM)),\
        "Shape of matrix{0} doesn't match predefined shape ({1},{2})"\
        .format(predictionMatrix.shape,ROW_NUM, COL_NUM)
    except Exception as e:
        print(e)

In [91]:
testPopularityRecSys()

calculated 0 users
calculated 100 users
calculated 200 users
calculated 300 users
calculated 400 users
calculated 500 users
calculated 600 users
calculated 700 users
calculated 800 users
calculated 900 users


### User Average Based Recommendation

In [92]:
def testUserAverRecSys(validation_df=validation_df, BaseLineRecSys = BaseLineRecSys):
    useraverage_recsys = BaseLineRecSys('average_user_rating')
    try:
        useraverage_recsys.predict_all(validation_df, ROW_NUM, COL_NUM)
    except:
        print('useraverage function has error')
        return
    try:
        predictionMatrix = useraverage_recsys.getModel()
        assert(predictionMatrix.shape == (ROW_NUM, COL_NUM)),\
        "Shape of matrix{0} doesn't match predefined shape ({1},{2})"\
        .format(predictionMatrix.shape,ROW_NUM, COL_NUM)
    except Exception as e:
        print(e)

In [93]:
testPopularityRecSys()

calculated 0 users
calculated 100 users
calculated 200 users
calculated 300 users
calculated 400 users
calculated 500 users
calculated 600 users
calculated 700 users
calculated 800 users
calculated 900 users


## Similary Based Recommendation Systems

### Euclidean Similarity Function

In [94]:
def testEuclidean(validation_df=validation_df, getMatrix=CrossValidation.getMatrix):
    matrix = getMatrix(validation_df, ROW_NUM, COL_NUM, RATING_COL)
    try:
        sim_matrix = SimBasedRecSys.euclidean(matrix)
        assert(sim_matrix.shape == (ROW_NUM, ROW_NUM)),\
        "Shape of matrix{0} doesn't match predefined shape ({1},{2})"\
        .format(sim_matrix.shape,ROW_NUM,ROW_NUM)
        assert(np.any(sim_matrix <= 1)),\
               "Exist similarity value that is not less or equal to 1."
    except Exception as e:
        print(e)        

In [95]:
testEuclidean()

### Customized Similarity Function (test somethingelse function)

In [96]:
def testCustomizedSim(validation_df=validation_df, getMatrix=CrossValidation.getMatrix):
    matrix = getMatrix(validation_df, ROW_NUM, COL_NUM, RATING_COL)
    try:
        sim_matrix = SimBasedRecSys.somethingelse(matrix)
        assert(sim_matrix.shape == (ROW_NUM, ROW_NUM)),\
        "Shape of matrix{0} doesn't match predefined shape ({1},{2})"\
        .format(sim_matrix.shape,ROW_NUM,ROW_NUM)
        assert(np.any(sim_matrix <= 1)),\
               "Exist similarity value that is not less or equal to 1."
    except Exception as e:
        print(e) 

In [97]:
testCustomizedSim()

### User-User Similarity Based Recommendation System

In [98]:
def testUUSimBasedRecSys(validation_df=validation_df, dataPreprocessor=dataPreprocessor):
    try:
        user_cosine_recsys = SimBasedRecSys('user','cosine', dataPreprocessor)
    except:
        print("Framework error, please contact TA if you see this.")
        return
    
    try:
        user_cosine_recsys.predict_all(validation_df, ROW_NUM, COL_NUM)
        predictionMatrix = user_cosine_recsys.getModel()
        assert(predictionMatrix.shape == (ROW_NUM, COL_NUM)),\
        "Shape of matrix{0} doesn't match predefined shape ({1},{2})"\
        .format(predictionMatrix.shape,ROW_NUM, COL_NUM)
    except Exception as e:
        print(e)

In [99]:
testUUSimBasedRecSys()

### Item-Item Similarity Based Recommendation System

In [100]:
def testIISimBasedRecSys(validation_df=validation_df, dataPreprocessor=dataPreprocessor):
    try:
        user_cosine_recsys = SimBasedRecSys('item','cosine', dataPreprocessor)
    except:
        print("Framework error, please contact TA if you see this.")
        return
    
    try:
        user_cosine_recsys.predict_all(validation_df, ROW_NUM, COL_NUM)
        predictionMatrix = user_cosine_recsys.getModel()
        assert(predictionMatrix.shape == (ROW_NUM, COL_NUM)),\
        "Shape of matrix{0} doesn't match predefined shape ({1},{2})"\
        .format(predictionMatrix.shape,ROW_NUM, COL_NUM)
    except Exception as e:
        print(e)

In [101]:
testIISimBasedRecSys()

# Refrences
Collaborated with Kamil Yilaci